In [ ]:
# Now that the Financial Data and Prices Data files have been created,
# merge them into one file and calculate the various items needed for analysis
# Open the CompanyData database
# In the Prices file, calculate the forward 3 month return
# Do an inner join on the two files
# calculate all the ratios for testing

In [1]:
import pandas as pd
import requests
import json
import datetime
import time
from dateutil.relativedelta import relativedelta
import math
import numpy as np

import sqlalchemy
from sqlalchemy.types import Integer, Text, String, Float, Date
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
# Turns out that I should have created Primary Keys for the two tables when I first created them
# I will do that first
database_name = "../data/CompanyData.sqlite"
engine = create_engine(f"sqlite:///{database_name}", echo=False)
inspector = inspect(engine)

In [ ]:
# Verify the list of columns in the 'findata' table
columns = inspector.get_columns('findata')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Verify the list of columns in the 'pricedata' table
columns = inspector.get_columns('pricedata')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Create a "FinData2" table, all the same data as 'findata', but add a primary key of "monthend_date" and "ticker"
create_FinData_sql = 'CREATE TABLE "FinData2" (' + \
    'idx BIGINT, ' + \
    'monthend_date DATE, ' + \
    'ticker VARCHAR(8), ' + \
    'cash FLOAT, ' + \
    'st_debt FLOAT, ' + \
    'lt_debt FLOAT, ' + \
    'equity FLOAT, ' + \
    'revenue FLOAT, ' + \
    'ebit FLOAT, ' + \
    'net_income FLOAT, ' + \
    'basic_eps FLOAT, ' + \
    'diluted_eps FLOAT, ' + \
    'shares FLOAT, ' + \
    'qtr_date DATE, ' + \
    'PRIMARY KEY (monthend_date, ticker));'

engine.execute(create_FinData_sql)

sql_stmt = "INSERT INTO FinData2 SELECT * FROM findata"
engine.execute(sql_stmt)

In [ ]:
# Create a "PriceData2" table, all the same data as 'pricedata', but add a primary key of "monthend_date" and "ticker"
create_PriceData_sql = 'CREATE TABLE "PriceData2" (' + \
    'idx BIGINT, ' + \
    'monthend_date DATE, ' + \
    'ticker VARCHAR(8), ' + \
    'price FLOAT, ' + \
    'wealth_index FLOAT, ' + \
    'PRIMARY KEY (monthend_date, ticker));'

engine.execute(create_PriceData_sql)

sql_stmt = "INSERT INTO PriceData2 SELECT * FROM pricedata"
engine.execute(sql_stmt)

In [3]:
# Read the PriceData2 table into a Pandas Dataframe
prices_df = pd.read_sql_query("SELECT * FROM PriceData2", engine)
prices_df.head(12)

,idx,monthend_date,ticker,price,wealth_index
0,0,2019-09-30,A,77.97,77.9700
1,1,2019-08-31,A,71.11,71.1100
2,2,2019-07-31,A,69.41,69.4100
3,3,2019-06-30,A,74.67,74.5060
4,4,2019-05-31,A,67.05,66.9027
5,5,2019-04-30,A,78.50,78.3276
6,6,2019-03-31,A,80.38,80.0398
7,7,2019-02-28,A,79.44,79.1038
8,8,2019-01-31,A,76.05,75.7281
9,9,2018-12-31,A,67.46,67.1745


In [4]:
# Create another dataframe with monthend_date and wealth_index
prices2_df = prices_df[['monthend_date', 'ticker', 'wealth_index']].copy()

# This will become the wealth_index at the end of the period, so change the name of wealth_index
prices2_df.rename(columns={'wealth_index': 'end_wealth_index'}, inplace=True)
prices2_df.head(12)

,monthend_date,ticker,end_wealth_index
0,2019-09-30,A,77.9700
1,2019-08-31,A,71.1100
2,2019-07-31,A,69.4100
3,2019-06-30,A,74.5060
4,2019-05-31,A,66.9027
5,2019-04-30,A,78.3276
6,2019-03-31,A,80.0398
7,2019-02-28,A,79.1038
8,2019-01-31,A,75.7281
9,2018-12-31,A,67.1745


In [5]:
# Since we are doing quarterly rebalancing, the end_wealth_index matches up with a start date 1 months prior
# Subtract 1 month from the monthend_date, to get the beginning of the performance measurement period

# Create a function to subtract "n" months from a date
def subtract_months (start_date, n_months):
    temp_year = int(start_date[0:4])
    temp_month = int(start_date[5:7]) - n_months
    if (temp_month == 0):
        new_date = datetime.datetime (temp_year-1, 12, 31) 
    elif (temp_month < 1):
        temp_year = temp_year - 1
        temp_month = temp_month + 12
        new_date = datetime.datetime (temp_year, temp_month + 1, 1) - relativedelta(days=1)
    else:
        new_date = datetime.datetime (temp_year, temp_month + 1, 1) - relativedelta(days=1)
    return new_date.strftime("%Y-%m-%d")
            
# Now run that function on the monthend_date, creating a new column called ending_date
prices2_df['starting_date'] = [subtract_months (i,1) for i in prices2_df['monthend_date']]
prices2_df.head(12)                                    

,monthend_date,ticker,end_wealth_index,starting_date
0,2019-09-30,A,77.9700,2019-08-31
1,2019-08-31,A,71.1100,2019-07-31
2,2019-07-31,A,69.4100,2019-06-30
3,2019-06-30,A,74.5060,2019-05-31
4,2019-05-31,A,66.9027,2019-04-30
5,2019-04-30,A,78.3276,2019-03-31
6,2019-03-31,A,80.0398,2019-02-28
7,2019-02-28,A,79.1038,2019-01-31
8,2019-01-31,A,75.7281,2018-12-31
9,2018-12-31,A,67.1745,2018-11-30


In [6]:
# Now, delete the column named "monthend_date"
prices2_df.drop(columns=['monthend_date'], inplace=True)
prices2_df.head(12)

,ticker,end_wealth_index,starting_date
0,A,77.9700,2019-08-31
1,A,71.1100,2019-07-31
2,A,69.4100,2019-06-30
3,A,74.5060,2019-05-31
4,A,66.9027,2019-04-30
5,A,78.3276,2019-03-31
6,A,80.0398,2019-02-28
7,A,79.1038,2019-01-31
8,A,75.7281,2018-12-31
9,A,67.1745,2018-11-30


In [7]:
# Change the name of "staring_date" to "monthend_date"
prices2_df.rename(columns={'starting_date': 'monthend_date'}, inplace=True)
prices2_df.head(12)

,ticker,end_wealth_index,monthend_date
0,A,77.9700,2019-08-31
1,A,71.1100,2019-07-31
2,A,69.4100,2019-06-30
3,A,74.5060,2019-05-31
4,A,66.9027,2019-04-30
5,A,78.3276,2019-03-31
6,A,80.0398,2019-02-28
7,A,79.1038,2019-01-31
8,A,75.7281,2018-12-31
9,A,67.1745,2018-11-30


In [8]:
# Now, do an inner join of "prices_df" and "prices2_df" on "ticker" and "monthend_date"
prices3_df = pd.merge(prices_df, prices2_df,  how='inner', left_on=['ticker','monthend_date'], right_on = ['ticker','monthend_date'])
prices3_df.drop(columns=['idx'], inplace=True)
prices3_df.head(12)

,monthend_date,ticker,price,wealth_index,end_wealth_index
0,2019-08-31,A,71.11,71.1100,77.9700
1,2019-07-31,A,69.41,69.4100,71.1100
2,2019-06-30,A,74.67,74.5060,69.4100
3,2019-05-31,A,67.05,66.9027,74.5060
4,2019-04-30,A,78.50,78.3276,66.9027
5,2019-03-31,A,80.38,80.0398,78.3276
6,2019-02-28,A,79.44,79.1038,80.0398
7,2019-01-31,A,76.05,75.7281,79.1038
8,2018-12-31,A,67.46,67.1745,75.7281
9,2018-11-30,A,72.35,71.8661,67.1745


In [9]:
# Calculate the total return from the beginning "wealth_index" to the ending "end_wealth_index"
prices3_df['total_return'] = np.maximum (-99.9, (prices3_df['end_wealth_index'] / prices3_df['wealth_index'] - 1) * 100)
prices3_df.head(12)

,monthend_date,ticker,price,wealth_index,end_wealth_index,total_return
0,2019-08-31,A,71.11,71.1100,77.9700,9.647026
1,2019-07-31,A,69.41,69.4100,71.1100,2.449215
2,2019-06-30,A,74.67,74.5060,69.4100,-6.839718
3,2019-05-31,A,67.05,66.9027,74.5060,11.364713
4,2019-04-30,A,78.50,78.3276,66.9027,-14.586046
5,2019-03-31,A,80.38,80.0398,78.3276,-2.139186
6,2019-02-28,A,79.44,79.1038,80.0398,1.183255
7,2019-01-31,A,76.05,75.7281,79.1038,4.457658
8,2018-12-31,A,67.46,67.1745,75.7281,12.733403
9,2018-11-30,A,72.35,71.8661,67.1745,-6.528252


In [10]:
# Sort the dataframe by ticker and ascending date
prices3_df.sort_values(['ticker', 'monthend_date'], inplace=True)
prices3_df.head(12)

,monthend_date,ticker,price,wealth_index,end_wealth_index,total_return
236,1999-12-31,A,77.313,47.8870,40.9962,-14.389709
235,2000-01-31,A,66.188,40.9962,64.3396,56.940399
234,2000-02-29,A,103.875,64.3396,64.4171,0.120455
233,2000-03-31,A,104.000,64.4171,54.8939,-14.783652
232,2000-04-30,A,88.625,54.8939,45.6029,-16.925378
231,2000-05-31,A,73.625,45.6029,45.6804,0.169945
230,2000-06-30,A,73.750,45.6804,25.2403,-44.745887
229,2000-07-31,A,40.750,25.2403,37.7831,49.693546
228,2000-08-31,A,61.000,37.7831,30.3116,-19.774714
227,2000-09-30,A,48.938,30.3116,28.6857,-5.363953


In [11]:
# Read the FinData2 table into a Pandas Dataframe
financial_df = pd.read_sql_query("SELECT * FROM FinData2", engine)
financial_df.drop(columns=['idx'], inplace=True)
financial_df.head(12)

,monthend_date,ticker,cash,st_debt,lt_debt,equity,revenue,ebit,net_income,basic_eps,diluted_eps,shares,qtr_date
0,2014-12-31,A,2.218000e+09,0.0,1.663000e+09,5.301000e+09,4.048000e+09,419000000.0,549000000.0,1.65,1.62,3.327273e+08,2014-10-31
1,2015-01-31,A,2.218000e+09,0.0,1.663000e+09,5.301000e+09,4.048000e+09,419000000.0,549000000.0,1.65,1.62,3.327273e+08,2014-10-31
2,2015-02-28,A,2.218000e+09,0.0,1.663000e+09,5.301000e+09,4.048000e+09,419000000.0,549000000.0,1.65,1.62,3.327273e+08,2014-10-31
3,2015-03-31,A,2.118000e+09,0.0,1.658000e+09,4.237000e+09,4.066000e+09,410000000.0,417000000.0,1.25,1.23,3.336000e+08,2015-01-31
4,2015-04-30,A,2.118000e+09,0.0,1.658000e+09,4.237000e+09,4.066000e+09,410000000.0,417000000.0,1.25,1.23,3.336000e+08,2015-01-31
5,2015-05-31,A,2.118000e+09,0.0,1.658000e+09,4.237000e+09,4.066000e+09,410000000.0,417000000.0,1.25,1.23,3.336000e+08,2015-01-31
6,2015-06-30,A,2.197000e+09,0.0,1.656000e+09,4.158000e+09,4.041000e+09,423000000.0,365000000.0,1.10,1.08,3.318182e+08,2015-04-30
7,2015-07-31,A,2.197000e+09,0.0,1.656000e+09,4.158000e+09,4.041000e+09,423000000.0,365000000.0,1.10,1.08,3.318182e+08,2015-04-30
8,2015-08-31,A,2.197000e+09,0.0,1.656000e+09,4.158000e+09,4.041000e+09,423000000.0,365000000.0,1.10,1.08,3.318182e+08,2015-04-30
9,2015-09-30,A,2.075000e+09,0.0,1.655000e+09,4.103000e+09,4.046000e+09,436000000.0,329000000.0,0.99,0.98,3.323232e+08,2015-07-31


In [12]:
# Now merge the financial data
master_df = pd.merge(prices3_df, financial_df,  how='inner', left_on=['ticker','monthend_date'], right_on = ['ticker','monthend_date'])
master_df.head(12)

,monthend_date,ticker,price,wealth_index,end_wealth_index,total_return,cash,st_debt,lt_debt,equity,revenue,ebit,net_income,basic_eps,diluted_eps,shares,qtr_date
0,2014-12-31,A,40.94,39.0850,36.1469,-7.517206,2.218000e+09,0.0,1.663000e+09,5.301000e+09,4.048000e+09,419000000.0,549000000.0,1.65,1.62,3.327273e+08,2014-10-31
1,2015-01-31,A,37.77,36.1469,40.3961,11.755365,2.218000e+09,0.0,1.663000e+09,5.301000e+09,4.048000e+09,419000000.0,549000000.0,1.65,1.62,3.327273e+08,2014-10-31
2,2015-02-28,A,42.21,40.3961,39.8624,-1.321167,2.218000e+09,0.0,1.663000e+09,5.301000e+09,4.048000e+09,419000000.0,549000000.0,1.65,1.62,3.327273e+08,2014-10-31
3,2015-03-31,A,41.55,39.8624,39.6897,-0.433240,2.118000e+09,0.0,1.658000e+09,4.237000e+09,4.066000e+09,410000000.0,417000000.0,1.25,1.23,3.336000e+08,2015-01-31
4,2015-04-30,A,41.37,39.6897,39.5170,-0.435125,2.118000e+09,0.0,1.658000e+09,4.237000e+09,4.066000e+09,410000000.0,417000000.0,1.25,1.23,3.336000e+08,2015-01-31
5,2015-05-31,A,41.19,39.5170,37.1057,-6.101931,2.118000e+09,0.0,1.658000e+09,4.237000e+09,4.066000e+09,410000000.0,417000000.0,1.25,1.23,3.336000e+08,2015-01-31
6,2015-06-30,A,38.58,37.1057,39.3851,6.142992,2.197000e+09,0.0,1.656000e+09,4.158000e+09,4.041000e+09,423000000.0,365000000.0,1.10,1.08,3.318182e+08,2015-04-30
7,2015-07-31,A,40.95,39.3851,34.9224,-11.330935,2.197000e+09,0.0,1.656000e+09,4.158000e+09,4.041000e+09,423000000.0,365000000.0,1.10,1.08,3.318182e+08,2015-04-30
8,2015-08-31,A,36.31,34.9224,33.1139,-5.178625,2.197000e+09,0.0,1.656000e+09,4.158000e+09,4.041000e+09,423000000.0,365000000.0,1.10,1.08,3.318182e+08,2015-04-30
9,2015-09-30,A,34.33,33.1139,36.4224,9.991273,2.075000e+09,0.0,1.655000e+09,4.103000e+09,4.046000e+09,436000000.0,329000000.0,0.99,0.98,3.323232e+08,2015-07-31


In [13]:
# List the column names
list (master_df)

['monthend_date',
 'ticker',
 'price',
 'wealth_index',
 'end_wealth_index',
 'total_return',
 'cash',
 'st_debt',
 'lt_debt',
 'equity',
 'revenue',
 'ebit',
 'net_income',
 'basic_eps',
 'diluted_eps',
 'shares',
 'qtr_date']

In [14]:
# Add some criteria to the dataframe for testing
master_df['price_earnings'] = master_df['price'] / master_df['diluted_eps']

# As all quants know, earnings yield is really the item tested, as it handles the transition from
# positive to negative earnings in a linear fashion, unlike P/E which goes from positive infinity to negative infinity
# as earnings go from positive to negative
master_df['earnings_yield'] = master_df['diluted_eps'] / master_df['price'] * 100

# Price/Book and Book Yield
master_df['book_value'] = master_df['equity'] / master_df ['shares']
master_df['price_book'] = master_df['price'] / master_df['book_value']
master_df['book_yield'] = master_df["book_value"] / master_df["price"] * 100

# Market Capitalization of the equity
master_df['market_cap'] = master_df['price'] * master_df['shares']

# Net Debt to Capital
master_df['net_debt'] = master_df["st_debt"] + master_df["lt_debt"] - master_df["cash"]
master_df['net_debt_capital'] = master_df["net_debt"] / (master_df["net_debt"] + master_df["equity"]) * 100

# Enterprise Value to Revenue and Enterprise Value to EBIT
master_df['enterprise_value'] = master_df['net_debt'] + master_df['market_cap']

master_df['ev_revenue'] = master_df['enterprise_value'] / master_df['revenue']
master_df['revenue_ev'] = master_df['revenue'] / master_df['enterprise_value']

master_df['ev_ebit'] = master_df['enterprise_value'] / master_df['ebit']
master_df['ebit_ev'] = master_df['ebit'] / master_df['enterprise_value']


In [15]:
SP_fname = "../data/SP500.csv"
sp500_df = pd.read_csv(SP_fname, parse_dates = ["Quarter"])
sp500_df.rename(columns={'Symbol': 'ticker', 'Name':'name','Sector':'sector'}, inplace=True)
sp500_df.drop(columns=['Quarter'], inplace=True)
sp500_df.head()

,ticker,name,sector
0,A,Agilent Technologies Inc,Health Care
1,AAL,American Airlines Group,Industrials
2,AAP,Advance Auto Parts,Consumer Discretionary
3,AAPL,Apple Inc.,Information Technology
4,ABBV,AbbVie Inc.,Health Care


In [16]:
# Join the master_df and sp500_df on ticker
master2_df = pd.merge(master_df, sp500_df,  how='inner', on=['ticker'])
master2_df.head()

,monthend_date,ticker,price,wealth_index,end_wealth_index,total_return,cash,st_debt,lt_debt,equity,...,market_cap,net_debt,net_debt_capital,enterprise_value,ev_revenue,revenue_ev,ev_ebit,ebit_ev,name,sector
0,2014-12-31,A,40.94,39.0850,36.1469,-7.517206,2.218000e+09,0.0,1.663000e+09,5.301000e+09,...,1.362185e+10,-555000000.0,-11.694058,1.306685e+10,3.227978,0.309791,31.185810,0.032066,Agilent Technologies Inc,Health Care
1,2015-01-31,A,37.77,36.1469,40.3961,11.755365,2.218000e+09,0.0,1.663000e+09,5.301000e+09,...,1.256711e+10,-555000000.0,-11.694058,1.201211e+10,2.967418,0.336993,28.668518,0.034881,Agilent Technologies Inc,Health Care
2,2015-02-28,A,42.21,40.3961,39.8624,-1.321167,2.218000e+09,0.0,1.663000e+09,5.301000e+09,...,1.404442e+10,-555000000.0,-11.694058,1.348942e+10,3.332366,0.300087,32.194315,0.031061,Agilent Technologies Inc,Health Care
3,2015-03-31,A,41.55,39.8624,39.6897,-0.433240,2.118000e+09,0.0,1.658000e+09,4.237000e+09,...,1.386108e+10,-460000000.0,-12.178978,1.340108e+10,3.295888,0.303408,32.685561,0.030595,Agilent Technologies Inc,Health Care
4,2015-04-30,A,41.37,39.6897,39.5170,-0.435125,2.118000e+09,0.0,1.658000e+09,4.237000e+09,...,1.380103e+10,-460000000.0,-12.178978,1.334103e+10,3.281120,0.304774,32.539102,0.030732,Agilent Technologies Inc,Health Care


In [17]:
list(master2_df)

['monthend_date',
 'ticker',
 'price',
 'wealth_index',
 'end_wealth_index',
 'total_return',
 'cash',
 'st_debt',
 'lt_debt',
 'equity',
 'revenue',
 'ebit',
 'net_income',
 'basic_eps',
 'diluted_eps',
 'shares',
 'qtr_date',
 'price_earnings',
 'earnings_yield',
 'book_value',
 'price_book',
 'book_yield',
 'market_cap',
 'net_debt',
 'net_debt_capital',
 'enterprise_value',
 'ev_revenue',
 'revenue_ev',
 'ev_ebit',
 'ebit_ev',
 'name',
 'sector']

In [19]:
# To write this dataframe to an SQL table with any keys, do the following
# Use the "df.to_sql" to write the dataframe to an SQL table with a temporary name

# Delete the Temp table if it exists
# sql_stmt = "DROP TABLE Temp"
# engine.execute(sql_stmt)

# Now, write the dataframe to the SQL table
master2_df.to_sql('Temp', con=engine, if_exists='replace', index=False)

# Drop any existing "MasterData" table, the one that will contain the data and index
sql_stmt = "DROP TABLE MasterData"
engine.execute(sql_stmt)

# Create a "MasterData" table to hold all the data, and be sure to include the index definition
create_MasterData_sql = 'CREATE TABLE "MasterData" (' + \
    'monthend_date DATE, ' + \
    'ticker VARCHAR(8), ' + \
    'price FLOAT, ' + \
    'wealth_index FLOAT, ' + \
    'end_wealth_index FLOAT, ' + \
    'total_return FLOAT, ' + \
    'cash FLOAT, ' + \
    'st_debt FLOAT, ' + \
    'lt_debt FLOAT, ' + \
    'equity FLOAT, ' + \
    'revenue FLOAT, ' + \
    'ebit FLOAT, ' + \
    'net_income FLOAT, ' + \
    'basic_eps FLOAT, ' + \
    'diluted_eps FLOAT, ' + \
    'shares FLOAT, ' + \
    'qtr_date DATE, ' + \
    'price_earnings FLOAT, ' + \
    'earnings_yield FLOAT, ' + \
    'book_value FLOAT, ' + \
    'price_book FLOAT, ' + \
    'book_yield FLOAT, ' + \
    'market_cap FLOAT, ' + \
    'net_debt FLOAT, ' + \
    'net_debt_capital FLOAT, ' + \
    'enterprise_value FLOAT, ' + \
    'ev_revenue FLOAT, ' + \
    'revenue_ev FLOAT, ' + \
    'ev_ebit FLOAT, ' + \
    'ebit_ev FLOAT, ' + \
    'name VARCHAR, ' + \
    'sector VARCHAR, ' + \
    'PRIMARY KEY (monthend_date, ticker));'

# Create the table.  It will be empty
engine.execute(create_MasterData_sql)

# Now copy the data from the Temp table into the New table
sql_stmt = "INSERT INTO MasterData SELECT * FROM Temp"
engine.execute(sql_stmt)

# Delete the Temp table
sql_stmt = "DROP TABLE Temp"
engine.execute(sql_stmt)

In [20]:
# Test it
master3_df = pd.read_sql_query("SELECT * FROM MasterData", engine)
master3_df.head(12)

,monthend_date,ticker,price,wealth_index,end_wealth_index,total_return,cash,st_debt,lt_debt,equity,...,market_cap,net_debt,net_debt_capital,enterprise_value,ev_revenue,revenue_ev,ev_ebit,ebit_ev,name,sector
0,2014-12-31,A,40.94,39.0850,36.1469,-7.517206,2.218000e+09,0.0,1.663000e+09,5.301000e+09,...,1.362185e+10,-555000000.0,-11.694058,1.306685e+10,3.227978,0.309791,31.185810,0.032066,Agilent Technologies Inc,Health Care
1,2015-01-31,A,37.77,36.1469,40.3961,11.755365,2.218000e+09,0.0,1.663000e+09,5.301000e+09,...,1.256711e+10,-555000000.0,-11.694058,1.201211e+10,2.967418,0.336993,28.668518,0.034881,Agilent Technologies Inc,Health Care
2,2015-02-28,A,42.21,40.3961,39.8624,-1.321167,2.218000e+09,0.0,1.663000e+09,5.301000e+09,...,1.404442e+10,-555000000.0,-11.694058,1.348942e+10,3.332366,0.300087,32.194315,0.031061,Agilent Technologies Inc,Health Care
3,2015-03-31,A,41.55,39.8624,39.6897,-0.433240,2.118000e+09,0.0,1.658000e+09,4.237000e+09,...,1.386108e+10,-460000000.0,-12.178978,1.340108e+10,3.295888,0.303408,32.685561,0.030595,Agilent Technologies Inc,Health Care
4,2015-04-30,A,41.37,39.6897,39.5170,-0.435125,2.118000e+09,0.0,1.658000e+09,4.237000e+09,...,1.380103e+10,-460000000.0,-12.178978,1.334103e+10,3.281120,0.304774,32.539102,0.030732,Agilent Technologies Inc,Health Care
5,2015-05-31,A,41.19,39.5170,37.1057,-6.101931,2.118000e+09,0.0,1.658000e+09,4.237000e+09,...,1.374098e+10,-460000000.0,-12.178978,1.328098e+10,3.266351,0.306152,32.392644,0.030871,Agilent Technologies Inc,Health Care
6,2015-06-30,A,38.58,37.1057,39.3851,6.142992,2.197000e+09,0.0,1.656000e+09,4.158000e+09,...,1.280155e+10,-541000000.0,-14.957147,1.226055e+10,3.034037,0.329594,28.984741,0.034501,Agilent Technologies Inc,Health Care
7,2015-07-31,A,40.95,39.3851,34.9224,-11.330935,2.197000e+09,0.0,1.656000e+09,4.158000e+09,...,1.358795e+10,-541000000.0,-14.957147,1.304695e+10,3.228645,0.309727,30.843864,0.032421,Agilent Technologies Inc,Health Care
8,2015-08-31,A,36.31,34.9224,33.1139,-5.178625,2.197000e+09,0.0,1.656000e+09,4.158000e+09,...,1.204832e+10,-541000000.0,-14.957147,1.150732e+10,2.847641,0.351168,27.204062,0.036759,Agilent Technologies Inc,Health Care
9,2015-09-30,A,34.33,33.1139,36.4224,9.991273,2.075000e+09,0.0,1.655000e+09,4.103000e+09,...,1.140866e+10,-420000000.0,-11.403747,1.098866e+10,2.715931,0.368198,25.203341,0.039677,Agilent Technologies Inc,Health Care
